## key takeaway
- graphs as mental model for agents

### env setup

In [52]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

### search tools

In [53]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [55]:
search = TavilySearchResults(max_results=5)

In [58]:
results = search.invoke('Who is the prime minister of Singapore?')

In [62]:
results[0]

{'url': 'https://www.pmo.gov.sg/The-Cabinet/Mr-LEE-Hsien-Loong',
 'content': "Biodata\nCareer\n2004 -\nPrime Minister\nChairman, Research, Innovation and Enterprise Council (RIEC)\nChairman, Government of Singapore Investment Corporation (2011)\n2001 - 2007\nMinister for Finance\n1998 - 2004\nChairman, Monetary Authority of Singapore\n1990 - 2004\nDeputy Prime Minister\n1987 - 1992\nMinister for Trade and Industry\n1987 - 1990\nSecond Minister for Defence\n1984 - 1987\nMinister of State for Ministry of Trade and Industry &\nMinistry of Defence\n2004 -\nSecretary-General, People's Action Party\n1992 - 2004\nFirst Assistant Secretary-General, People's Action Party\n1989 - 1992\nSecond Assistant Secretary-General, People's Action Party\n1986 - 1989\nMember, Central Executive Committee of the People's Action Party\n1984 -\nMember of Parliament\n(First elected in 1984, and re-elected in 1988, 1991, 1997, 2001, 2006 and 2011)\n1982 - 1984\nChief of Staff of the General Staff\n1983 - 1984\nDi

- langchain is good for these kind of convenient abstractions. easy to experiment with integrations like search APIs.
- but you can easily end up in a deep black hole if you try to trace the abstraction. see source code examples [here](https://api.python.langchain.com/en/latest/_modules/langchain_community/tools/tavily_search/tool.html#TavilySearchResults) and [here](https://api.python.langchain.com/en/latest/_modules/langchain_core/tools.html#BaseTool)

### retriever 

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://learn.microsoft.com/en-us/azure/search/semantic-search-overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [10]:
retriever.invoke("limitations of semantic ranking?")

[Document(page_content="What semantic ranking can't do is rerun the query over the entire corpus to find semantically relevant results. Semantic ranking reranks the existing result set, consisting of the top 50 results as scored by the default ranking algorithm. Furthermore, semantic ranking can't create new information or strings. Captions and answers are extracted verbatim from your content so if the results don't include answer-like text, the language models won't produce one.\nAlthough semantic ranking isn't beneficial in every scenario, certain content can benefit significantly from its capabilities. The language models in semantic ranking work best on searchable content that is information-rich and structured as prose. A knowledge base, online documentation, or documents that contain descriptive content see the most gains from semantic ranking capabilities.", metadata={'source': 'https://learn.microsoft.com/en-us/azure/search/semantic-search-overview', 'title': 'Semantic ranking 

### tool binding

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "azure_ai_search",
    "Search for information about Azure Semantic Search. For any questions about Azure Semantic Search, you must use this tool!",
)

tools = [search, retriever_tool]

- abstractions are good, but you quickly realise that you don't have a good overview of the expected inputs/outputs from each of these tools... hard to control and debug down the line

In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [14]:
from langchain_core.messages import HumanMessage
# ummm is this really necessary? example of abstraction that makes you go hmm...

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [15]:
model_with_tools = model.bind_tools(tools)

In [16]:
from pprint import pprint

In [17]:
def invoke_model_with_tools(message: str, model_with_tools=model_with_tools) -> None:

    response = model_with_tools.invoke([HumanMessage(content=message)])

    print(f"ContentString: {response.content}")
    print(f"ToolCalls: {response.tool_calls}")
    pprint(response.dict())

In [18]:
invoke_model_with_tools('hi')

ContentString: Hello! How can I assist you today?
ToolCalls: []
{'additional_kwargs': {},
 'content': 'Hello! How can I assist you today?',
 'example': False,
 'id': 'run-76868228-568a-4dc8-8bbc-ada63d6e80e0-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'logprobs': None,
                       'model_name': 'gpt-3.5-turbo-0125',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 10,
                                       'prompt_tokens': 130,
                                       'total_tokens': 140}},
 'tool_calls': [],
 'type': 'ai'}


In [19]:
invoke_model_with_tools("what's the weather in singapore")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Singapore'}, 'id': 'call_5agZy6GguC2KoM7PVFVi2XQh'}]
{'additional_kwargs': {'tool_calls': [{'function': {'arguments': '{"query":"weather '
                                                                 'in '
                                                                 'Singapore"}',
                                                    'name': 'tavily_search_results_json'},
                                       'id': 'call_5agZy6GguC2KoM7PVFVi2XQh',
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-2c2294ab-67a7-4997-a59e-9c17394b1faf-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'logprobs': None,
                       'model_name': 'gpt-3.5-turbo-0125',
                       'system_fingerprint': None,
                       'token_usage': {'complet

#### what would this have looked like if you used the api directly - [taken from openai docs directly]
```python
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())
```

In [20]:
def invoke_agent(message):
    
    from langgraph.prebuilt import chat_agent_executor

    agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)
    
    response = agent_executor.invoke({"messages": [HumanMessage(content=message)]})
    
    return response['messages']

okay let's go down the rabbit [hole](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/chat_agent_executor.py) -- now we enter the graph frameworks (see `create_tool_calling_executor`)

In [21]:
invoke_agent('hi')

[HumanMessage(content='hi', id='4cb04639-616f-452b-b93e-287c56d19420'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 130, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-75f31d16-6eea-42b8-af2e-30256aa3f7c2-0')]

In [23]:
result = invoke_agent('what are the limitations of azure semantic search')
print(result[-1].content)

In [29]:
result = invoke_agent('who is wei boon goh')
print(result[-1].content)

# oops outdated information!
# btw for many search apis nowadays, you can filter for published date to overcome this problem

Wei Boon Goh is the Chief Executive at the Centre for Strategic Infocomm Technologies (CSIT). He will be appointed as the Chief Executive (CE) of the Government Technology Agency (GovTech) with effect from 1 June 2023, succeeding Mr. Kok Ping Soon. Wei Boon Goh has a background in technology and leadership. You can find more information about him on his [LinkedIn profile](https://sg.linkedin.com/in/weiboon).


See [langsmith](https://smith.langchain.com) also -- just a toy really